In [5]:
import pandas as pd
import json 
import yaml
from pymongo import MongoClient
from sklearn.model_selection import train_test_split

def combine_json(files):
    combined_data = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as fin:
            # Load data from the file and append it to the combined list
            data = json.load(fin)
            combined_data.extend(data)
    return combined_data


initial_files = [f'../data/previous_dbs/final_db{i}.json' for i in range(1,5)]
initial_db = combine_json(initial_files)
cleaned_db_file = ["../data/cleaned_articles.json"]
cleaned_db = combine_json(cleaned_db_file)
phrase_bert_file = ['../data/final_db_pb_tags.json']
phrase_bert_db = combine_json(phrase_bert_file)

for initial in initial_db:
    initial_id = initial['id']
    for clean in cleaned_db:
        if clean['_id'] == initial_id:
            initial['article_url'] = clean['_source']['content_metadata']['article_url']
            try:
                initial['st_id'] = clean['_source']['identification']['drupal']['id']
            except KeyError:
                initial['st_id'] = clean['_source']['identification']['cue']['id']
    for pb in phrase_bert_db:
        if pb['id'] == initial_id:
            initial['phrase_Bert_tags_embeddings'] = pb['phrase_Bert_tags_embeddings']
for initial in initial_db:
    initial.pop('id', None)
filtered_initial = [d for d in initial_db if all(value is not None for value in d.values())]
def convert_lists_to_string(dicts, keys_to_convert):
    """
    Convert lists to string representations for specified keys in each dictionary.
    
    :param dicts: List of dictionaries to process.
    :param keys_to_convert: List of keys whose values need to be converted from lists to strings.
    :return: List of dictionaries with the specified keys' values converted to strings.
    """
    for d in dicts:
        for key in keys_to_convert:
            if key in d and isinstance(d[key], list):
                try:
                    # Convert the list to a string representation
                    d[key] = json.dumps(d[key], ensure_ascii=False)
                except (TypeError, ValueError) as e:
                    pass  # Handle or log the error as needed
    return dicts

keys_to_convert = ['Title_embeddings', 'tags_embeddings', 'phrase_Bert_tags_embeddings']
filtered_initial = convert_lists_to_string(filtered_initial, keys_to_convert)
filtered_initial[0]
final_df = pd.DataFrame(filtered_initial)
df_train, df_test = train_test_split(final_df, test_size=7)

# Load configuration
with open('../gradio_config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Extract MongoDB configuration
mongo_uri = config['database']['uri']
db_name = config['database']['name']
train_collection_name = config['database']['train_collection']
test_collection_name = config['database']['test_collection']

# Connect to MongoDB
client = MongoClient(mongo_uri)
db = client[db_name]
train_collection = db[train_collection_name]
test_collection = db[test_collection_name]

# Delete all documents in the collections
train_collection.delete_many({})
test_collection.delete_many({})

# Convert DataFrames to list of dictionaries
train_records = df_train.to_dict(orient='records')
test_records = df_test.to_dict(orient='records')

# Insert records into MongoDB
train_collection.insert_many(train_records)
test_collection.insert_many(test_records)

print("New Data successfully deleted and inserted into MongoDB")


Data successfully inserted into MongoDB


In [6]:
import os
import ast
import sys
import json
import yaml
import re
from json import JSONDecodeError
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from pymongo import MongoClient


import gradio as gr
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster

# Import libraries for working with language models and Google Gemini
from langchain_core.prompts import PromptTemplate
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Load environment variables
load_dotenv()
GEMINI_KEY = os.environ.get('GEMINI_KEY')
genai.configure(api_key=GEMINI_KEY)

# Normally where to do this? (in which function?)
with open("../gradio_config.yaml", "r") as config_file:
    config = yaml.safe_load(config_file)

# Initialise mongo client.
mongo_client = MongoClient(config["database"]["uri"])


In [7]:
def load_mongo_test():
    print("Fetching article data from MongoDB...\n")
    # Connect to the MongoDB client
    try:
        db = mongo_client[config["database"]["name"]]
        train_docs = db[config["database"]["test_collection"]].find()
        print("Data successfully fetched from MongoDB\n")
    except Exception as error: 
        print(f"Unable to fetch data from MongoDB. Check your connection the database...\n")
        print(f"ERROR: {error}\n")
        sys.exit()
    return train_docs

test_database = load_mongo_test()

Fetching article data from MongoDB...

Data successfully fetched from MongoDB



In [9]:
def count_test_length(test_database):
    count = 0
    for doc in test_database:
        count += 1
    return count

7
